In [1]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline, BitsAndBytesConfig , CodeGenTokenizer 
from langchain.llms import HuggingFacePipeline 
from langchain import PromptTemplate, LLMChain
from transformers import AutoTokenizer , AutoModelForCausalLM
import torch 

In [2]:
quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)

In [3]:
tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/phi-2",
    trust_remote_code = True
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    torch_dtype=torch.float32,
    device_map = "auto",
    trust_remote_code = True,
    load_in_8bit=True,
    quantization_config=quantization_config
)
     


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# prompt = "Write a short summary about how to create a healthy lifestyle."
# with torch.no_grad():
#   token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
#   output_ids = model.generate(
#       token_ids.to(model.device),
#       max_new_tokens=512,
#       do_sample=True,
#       temperature = 0.1
#   )

# output = tokenizer.decode(output_ids[0][token_ids.size(1) :])

In [6]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=320,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2,
    do_sample=True
)
local_llm = HuggingFacePipeline(pipeline=pipe)

In [7]:
from langchain import PromptTemplate, LLMChain
template = """respond to the instruction below. behave like a chatbot 
and respond to the user. try to be helpful.
### Instruction:
{instruction}
Answer:"""
prompt = PromptTemplate(template=template, input_variables=["instruction"])

In [8]:
llm_chain = LLMChain(prompt=prompt,
                     llm=local_llm
                     )

In [9]:

question = """
Digital root is the recursive sum of all the digits in a number.
Write a function in python that given n, take the sum of the digits of n.
If that value has more than one digit, continue reducing in this way until a single-digit number is produced.
The input will be a non-negative integer. The output will be the Digital root.
Examples: 16  -->  1 + 6 = 7
"""
reponse = llm_chain.invoke(question)
print(reponse)

{'instruction': '\nDigital root is the recursive sum of all the digits in a number.\nWrite a function in python that given n, take the sum of the digits of n. If that value has more than one digit, continue reducing in this way until a single-digit number is produced. The input will be a non-negative integer. The output will be the Digital root.\nExamples\n   16  -->  1 + 6 = 7\n\n', 'text': ' 1\n"""#!/usr/bin/env python3\r\nimport sys\r\nfrom itertools import product as cartesian_product\r\n\r\nnumbers = [int(number) for number in open("input").read().splitlines()]\r\n\r\nfor i, j, k, l in list(cartesian_product(*[range(10)] * 4)) if (i + j + k + l == 2020):\r\n    print(f"{i}*{j}*{k}*{l}= {100 * i+ 10 * j+ 5 * k+l}")\r\n\r\n# -*- coding: utf-8 -*-\n##############################################\n# pySAP - Copyright © 2014-2018 Jean Bredin            #\n# Distributed under the terms of the CeCILL-B license,       #\n# as published by the C'}


In [11]:
print(reponse["text"])

 1
"""#!/usr/bin/env python3
import sys
from itertools import product as cartesian_product

numbers = [int(number) for number in open("input").read().splitlines()]

for i, j, k, l in list(cartesian_product(*[range(10)] * 4)) if (i + j + k + l == 2020):
    print(f"{i}*{j}*{k}*{l}= {100 * i+ 10 * j+ 5 * k+l}")

# -*- coding: utf-8 -*-
##############################################
# pySAP - Copyright © 2014-2018 Jean Bredin            #
# Distributed under the terms of the CeCILL-B license,       #
# as published by the C
